<a href="https://colab.research.google.com/github/MitziVite/machine_learning/blob/main/notebooks/starter_housing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [11]:
import pandas as pd

housing = pd.read_csv('https://raw.githubusercontent.com/byui-cse/cse450-course/master/data/housing.csv')

In [12]:
%pip install --user xgboost

import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
import xgboost as xgb

# Load dataset (replace with actual filename)
data = pd.read_csv('https://raw.githubusercontent.com/byui-cse/cse450-course/master/data/housing.csv')
# Drop the 'id' column as it is not a useful feature
data = data.drop(columns=['id'])

# Extract 'year_sold' from the 'date' column and drop the original 'date' column
data['year_sold'] = data['date'].str[:4].astype(int)
data = data.drop(columns=['date'])

# Select only the 7 most relevant features
selected_features = ['sqft_living', 'bedrooms', 'bathrooms', 'grade', 'condition', 'yr_built', 'zipcode']

X = data.drop(columns=['price'])

# X = data[selected_features]  # Features
y = data['price']  # Target

# Encode 'zipcode' as it is categorical
# if 'zipcode' in X.columns:
#     le = LabelEncoder()
#     X.loc[:, 'zipcode'] = LabelEncoder().fit_transform(X['zipcode'])

# Split the dataset into training (80%) and test (20%) sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Train the XGBoost model
xgb_model = xgb.XGBRegressor(
    objective='reg:squarederror',
    n_estimators=500,
    learning_rate=0.05,
    max_depth=6,
    subsample=0.8,
    colsample_bytree=0.8,
    random_state=42
)

xgb_model.fit(X_train, y_train)

# Compute R^2 scores for training and test sets
train_score = xgb_model.score(X_train, y_train)
test_score = xgb_model.score(X_test, y_test)

print(f"Train Score: {train_score:.4f}")
print(f"Test Score: {test_score:.4f}")


Train Score: 0.9772
Test Score: 0.8845


In [14]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder

# Load new dataset (for prediction)
new_data_path = "https://raw.githubusercontent.com/byui-cse/cse450-course/master/data/housing_holdout_test_mini.csv"
new_data = pd.read_csv(new_data_path)

# Extract year_sold from 'date' and drop 'id' & 'date'
new_data['year_sold'] = new_data['date'].str[:4].astype(int)
new_data = new_data.drop(columns=['id', 'date'])

# Ensure X_new has the same columns and order as the training data (X_train)
# Get the feature names from X_train (which was used to train the model)
training_features = X_train.columns.tolist()

# Create X_new by selecting columns that match training_features from new_data
# This ensures the features are aligned correctly for prediction.
X_new = new_data[training_features]

# Predict prices using trained model
predicted_prices = xgb_model.predict(X_new)

# Round predictions to the nearest whole number
predicted_prices = predicted_prices.round(0)

# Save predictions as a single-column CSV with header 'price'
results = pd.DataFrame({'price': predicted_prices})
results.to_csv("team8-module3-predictions.csv", index=False)

print("Predictions saved to team8-module3-predictions.csv")

Predictions saved to team8-module3-predictions.csv


In [ ]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder

# Load new dataset (for prediction)
new_data_path = "https://raw.githubusercontent.com/byui-cse/cse450-course/master/data/housing_holdout_test_mini.csv"
new_data = pd.read_csv(new_data_path)

# Extract 'year_sold' from 'date' and drop 'id' & 'date'
new_data['year_sold'] = new_data['date'].str[:4].astype(int)
new_data = new_data.drop(columns=['id', 'date'])

# Ensure X_new has the same columns and order as the training data (X_train)
# Get the feature names from X_train (which was used to train the model)
training_features = X_train.columns.tolist()

# Reindex new_data to match the training features, filling missing columns with 0 if necessary
# (though in this case, new_data is expected to have all the same columns as the original data)
X_new = new_data[training_features]

# Predict prices using the trained model
predicted_prices = xgb_model.predict(X_new)

# Round predictions to the nearest whole number
predicted_prices = predicted_prices.round(0)

# Save predictions as a single-column CSV with header 'price'
results = pd.DataFrame({'price': predicted_prices})
results.to_csv("team8-module3-predictions.csv", index=False)

print("Predictions saved to team8-module3-predictions.csv")